In [6]:
pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 20.3 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pygame
import random

pygame.init()

# Screen dimensions
SCREEN_WIDTH = 300
SCREEN_HEIGHT = 600
GRID_SIZE = 30

# Define the shapes of the tetriminos
SHAPES = [
    [[1, 1, 1, 1]],  # I
    [[1, 1], [1, 1]],  # O
    [[0, 1, 0], [1, 1, 1]],  # T
    [[1, 1, 0], [0, 1, 1]],  # S
    [[0, 1, 1], [1, 1, 0]],  # Z
    [[1, 1, 1], [1, 0, 0]],  # L
    [[1, 1, 1], [0, 0, 1]],  # J
]

# Colors for each shape
SHAPE_COLORS = [
    (0, 255, 255),  # Cyan (I)
    (255, 255, 0),  # Yellow (O)
    (128, 0, 128),  # Purple (T)
    (0, 255, 0),    # Green (S)
    (255, 0, 0),    # Red (Z)
    (255, 165, 0),  # Orange (L)
    (0, 0, 255),    # Blue (J)
]

# screen and game variables
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Tetris")
clock = pygame.time.Clock()
score = 0
grid = [[0] * (SCREEN_WIDTH // GRID_SIZE) for _ in range(SCREEN_HEIGHT // GRID_SIZE)]

# Tetrimino class
class Tetrimino:
    def __init__(self):
        self.shape = random.choice(SHAPES)
        self.color = SHAPE_COLORS[SHAPES.index(self.shape)]
        self.x = SCREEN_WIDTH // GRID_SIZE // 2 - len(self.shape[0]) // 2
        self.y = 0

    def rotate(self):
        self.shape = [list(row) for row in zip(*self.shape[::-1])]

    def move(self, dx, dy):
        self.x += dx
        self.y += dy

    def draw(self, screen):
        for y, row in enumerate(self.shape):
            for x, cell in enumerate(row):
                if cell:
                    pygame.draw.rect(screen, self.color, pygame.Rect(
                        (self.x + x) * GRID_SIZE, (self.y + y) * GRID_SIZE, GRID_SIZE, GRID_SIZE))

    def collides(self, dx=0, dy=0):
        for y, row in enumerate(self.shape):
            for x, cell in enumerate(row):
                if cell:
                    nx, ny = self.x + x + dx, self.y + y + dy
                    if nx < 0 or nx >= SCREEN_WIDTH // GRID_SIZE or ny >= SCREEN_HEIGHT // GRID_SIZE or grid[ny][nx]:
                        return True
        return False

# Game functions
def clear_lines():
    global score
    new_grid = [row for row in grid if any(cell == 0 for cell in row)]
    lines_cleared = len(grid) - len(new_grid)
    if lines_cleared > 0:
        score += lines_cleared * 10
        grid[:] = [[0] * (SCREEN_WIDTH // GRID_SIZE) for _ in range(lines_cleared)] + new_grid

def place_tetrimino(tetrimino):
    for y, row in enumerate(tetrimino.shape):
        for x, cell in enumerate(row):
            if cell:
                grid[tetrimino.y + y][tetrimino.x + x] = tetrimino.color

def draw_grid():
    for y, row in enumerate(grid):
        for x, cell in enumerate(row):
            if cell:
                pygame.draw.rect(screen, cell, pygame.Rect(x * GRID_SIZE, y * GRID_SIZE, GRID_SIZE, GRID_SIZE))

def draw_next_tetrimino(tetrimino):
    font = pygame.font.Font(None, 36)
    label = font.render("Next:", True, (255, 255, 255))
    screen.blit(label, (SCREEN_WIDTH + 10, 20))

    for y, row in enumerate(tetrimino.shape):
        for x, cell in enumerate(row):
            if cell:
                pygame.draw.rect(screen, tetrimino.color, pygame.Rect(
                    SCREEN_WIDTH + 10 + x * GRID_SIZE, 50 + y * GRID_SIZE, GRID_SIZE, GRID_SIZE))

def draw_score():
    font = pygame.font.Font(None, 36)
    label = font.render(f"Score: {score}", True, (255, 255, 255))
    screen.blit(label, (SCREEN_WIDTH + 10, 200))

# Main game loop
def main():
    global grid

    running = True
    current_tetrimino = Tetrimino()
    next_tetrimino = Tetrimino()
    drop_speed = 500
    drop_timer = pygame.USEREVENT + 1
    pygame.time.set_timer(drop_timer, drop_speed)

    while running:
        screen.fill((0, 0, 0))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT and not current_tetrimino.collides(dx=-1):
                    current_tetrimino.move(-1, 0)
                if event.key == pygame.K_RIGHT and not current_tetrimino.collides(dx=1):
                    current_tetrimino.move(1, 0)
                if event.key == pygame.K_DOWN and not current_tetrimino.collides(dy=1):
                    current_tetrimino.move(0, 1)
                if event.key == pygame.K_UP:
                    current_tetrimino.rotate()
                    if current_tetrimino.collides():
                        current_tetrimino.rotate()  # Undo the rotation if it collides
            if event.type == drop_timer:
                if not current_tetrimino.collides(dy=1):
                    current_tetrimino.move(0, 1)
                else:
                    place_tetrimino(current_tetrimino)
                    clear_lines()
                    current_tetrimino = next_tetrimino
                    next_tetrimino = Tetrimino()
                    if current_tetrimino.collides():
                        running = False

        draw_grid()
        current_tetrimino.draw(screen)
        draw_next_tetrimino(next_tetrimino)
        draw_score()
        pygame.display.flip()

        clock.tick(60)

    pygame.quit()

if __name__ == "__main__":
    main()